## Predict House Prices

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot  as plt
%matplotlib inline

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.model_selection import KFold

In [3]:
df = pd.read_csv('AmesHousing.tsv',  delimiter="\t")
df.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [4]:
# create function template for later update or use
def transform_features(df):
    return df

def select_features(df):
    return df[['Gr Liv Area','SalePrice']]

def train_and_test(df):
    train = df.iloc[:1460]
    test = df.iloc[1460:]
    
    ## You can use `pd.DataFrame.select_dtypes()` to specify column types
    ## and return only those columns as a data frame.
    numeric_train = train.select_dtypes(include=['integer','float'])
    numeric_test = test.select_dtypes(include=['integer','float'])
    
    ## You can use `pd.Series.drop()` to drop a value.
    features = numeric_train.columns.drop('SalePrice')
    lr = linear_model.LinearRegression()
    lr.fit(train[features], train['SalePrice'])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test['SalePrice'], predictions)
    rmse = np.sqrt(mse)
    
    return rmse

transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)
rmse

57088.25161263909

### Feature Engineering 

- Handle missing values:
  - All columns:
    - Drop any with 5% or more missing values for now.
  - Text columns:
    - Drop any with 1 or more missing values for now.
  - Numerical columns:
    - For columns with missing values, fill in with the most common value in that column

In [5]:
# Drop any with 5% or more missing values for now.
lenth = len(df)
null = df.isnull().sum()/lenth
drop_cols = list(null[null>=0.05].index)
df.drop(drop_cols, axis=1,inplace=True)

In [6]:
# Text columns:Drop any with 1 or more missing values for now
null = df.select_dtypes(include='object').isnull().sum()
drop_cols = list(null[null>0].index)
df.drop(drop_cols, axis=1,inplace=True)

In [7]:
# Numerical columns: For columns with missing values
# Fill in with the most common value in that column
num_missing = df.select_dtypes(include=['int', 'float']).isnull().sum()
fixable_cols = list(num_missing[num_missing>0].index)
fixable_value = df[fixable_cols].mode().to_dict(orient='records')[0]
df.fillna(fixable_value, inplace=True)

In [8]:
# Verify that every column has 0 missing values
df.isnull().sum().value_counts()

0    64
dtype: int64

What new features can we create, that better capture the information in some of the features?



In [9]:
years_sold = df['Yr Sold'] - df['Year Built']
years_sold[years_sold < 0]

2180   -1
dtype: int64

In [10]:
years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
years_since_remod[years_since_remod < 0]

1702   -1
2180   -2
2181   -1
dtype: int64

In [11]:
## Create new columns
df['Years Before Sale'] = years_sold
df['Years Since Remod'] = years_since_remod

## Drop rows with negative values for both of these new features
df = df.drop([1702, 2180, 2181], axis=0)

## No longer need original year columns
df = df.drop(["Year Built", "Year Remod/Add"], axis = 1)

- Drop columns that:
  - that aren't useful for ML
  - leak data about the final sale

In [12]:
## Drop columns that aren't useful for ML
df = df.drop(["PID", "Order"], axis=1)

## Drop columns that leak info about the final sale
df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)

Now. let's update functions

In [29]:
# create function template for later update or use
def transform_features(df):
    lenth = len(df)
    null = df.isnull().sum()/lenth
    drop_cols = list(null[null>=0.05].index)
    df.drop(drop_cols, axis=1,inplace=True)

    null = df.select_dtypes(include='object').isnull().sum()
    drop_cols = list(null[null>0].index)
    df.drop(drop_cols, axis=1,inplace=True)

    num_missing = df.select_dtypes(include=['int', 'float']).isnull().sum()
    fixable_cols = list(num_missing[num_missing>0].index)
    fixable_value = df[fixable_cols].mode().to_dict(orient='records')[0]
    df.fillna(fixable_value, inplace=True)

    years_sold = df['Yr Sold'] - df['Year Built']
    years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['Years Before Sale'] = years_sold
    df['Years Since Remod'] = years_since_remod

    df = df.drop([1702, 2180, 2181], axis=0)
    df = df.drop(["Year Built", "Year Remod/Add"], axis = 1)
    df = df.drop(["PID", "Order"], axis=1)
    df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)	
    return df

def select_features(df):
    return df[['Gr Liv Area','SalePrice']]

def train_and_test(df):
    train = df.iloc[:1460]
    test = df.iloc[1460:]
    
    ## You can use `pd.DataFrame.select_dtypes()` to specify column types
    ## and return only those columns as a data frame.
    numeric_train = train.select_dtypes(include=['integer','float'])
    numeric_test = test.select_dtypes(include=['integer','float'])
    
    ## You can use `pd.Series.drop()` to drop a value.
    features = numeric_train.columns.drop('SalePrice')
    lr = linear_model.LinearRegression()
    lr.fit(train[features], train['SalePrice'])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test['SalePrice'], predictions)
    rmse = np.sqrt(mse)
    
    return rmse

df = pd.read_csv('AmesHousing.tsv',  delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)
rmse

55275.36731241307

### Feature Selection

In [36]:
numerical_df  = transform_df.select_dtypes(include=['float','int'])
abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
abs_corr_coeffs

BsmtFin SF 2         0.006127
Misc Val             0.019273
3Ssn Porch           0.032268
Bsmt Half Bath       0.035875
Low Qual Fin SF      0.037629
Pool Area            0.068438
MS SubClass          0.085128
Overall Cond         0.101540
Screen Porch         0.112280
Kitchen AbvGr        0.119760
Enclosed Porch       0.128685
Bedroom AbvGr        0.143916
Bsmt Unf SF          0.182751
Lot Area             0.267520
2nd Flr SF           0.269601
Bsmt Full Bath       0.276258
Half Bath            0.284871
Open Porch SF        0.316262
Wood Deck SF         0.328183
BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
Years Since Remod    0.534985
Full Bath            0.546118
Years Before Sale    0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice            1.000000
Name: Sale

In [37]:
## Let's only keep columns with a correlation coefficient of larger than 0.4 (arbitrary, worth experimenting later!)
abs_corr_coeffs[abs_corr_coeffs > 0.4]
## Drop columns with less than 0.4 correlation with SalePrice
transform_df = transform_df.drop(abs_corr_coeffs[abs_corr_coeffs < 0.4].index, axis=1)


In [57]:
## Create a list of column names from documentation that are *meant* to be categorical
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

Which columns are currently numerical but need to be encoded as categorical instead (because the numbers don't have any semantic meaning) 

In [64]:
## Which categorical columns have we still carried with us? We'll test these


object_cols = list(transform_df.select_dtypes(include='object').columns)
num_cols = list(transform_df.select_dtypes(include=['int','float']).columns)
missing_cols = []
transform_cat_cols = object_cols

for i in num_cols: 
    if i in nominal_features:
        transform_cat_colsa.append(i)
        missing_cols.append(i)
        
# Test which numerical cols should be converted to categorical data
missing_cols
# all columns with more than 10 unique values, we will drop for now

object_unique = transform_df[transform_cat_cols].nunique()
transform_df.drop(object_unique[object_unique>10].index, axis=1, inplace=True)        

In [68]:
## Select just the remaining text columns and convert to categorical
text_cols = transform_df.select_dtypes(include=['object'])
for col in text_cols:
    transform_df[col] = transform_df[col].astype('category')
    
## Create dummy columns and add back to the dataframe!
dummy_df = pd.get_dummies(transform_df.select_dtypes(include=['category']))
transform_df = pd.concat([
    transform_df, 
    dummy_df
], axis=1).drop(text_cols,axis=1)

Update select_features() function

In [75]:
# create function template for later update or use
def transform_features(df):
    lenth = len(df)
    null = df.isnull().sum()/lenth
    drop_cols = list(null[null>=0.05].index)
    df.drop(drop_cols, axis=1,inplace=True)

    null = df.select_dtypes(include='object').isnull().sum()
    drop_cols = list(null[null>0].index)
    df.drop(drop_cols, axis=1,inplace=True)

    num_missing = df.select_dtypes(include=['int', 'float']).isnull().sum()
    fixable_cols = list(num_missing[num_missing>0].index)
    fixable_value = df[fixable_cols].mode().to_dict(orient='records')[0]
    df.fillna(fixable_value, inplace=True)

    years_sold = df['Yr Sold'] - df['Year Built']
    years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['Years Before Sale'] = years_sold
    df['Years Since Remod'] = years_since_remod

    df = df.drop([1702, 2180, 2181], axis=0)
    df = df.drop(["Year Built", "Year Remod/Add"], axis = 1)
    df = df.drop(["PID", "Order"], axis=1)
    df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)	
    return df

def select_features(df):
    numerical_df  = df.select_dtypes(include=['float','int'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < 0.4].index, axis=1)

    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                        "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                        "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                        "Misc Feature", "Sale Type", "Sale Condition"]
    object_cols = list(df.select_dtypes(include='object').columns)
    num_cols = list(df.select_dtypes(include=['int','float']).columns)
    missing_cols = []
    transform_cat_cols = object_cols

    for i in num_cols: 
        if i in nominal_features:
            transform_cat_colsa.append(i)
            missing_cols.append(i)
    
    object_unique = df[transform_cat_cols].nunique()
    df.drop(object_unique[object_unique>10].index, axis=1, inplace=True)

    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')

    dummy_df = pd.get_dummies(df.select_dtypes(include=['category']))
    df = pd.concat([
        df, 
        dummy_df
    ], axis=1).drop(text_cols,axis=1)
    
    return df

def train_and_test(df, k=0):
    
    numeric_df = df.select_dtypes(include=['integer','float'])
    features = numeric_df.columns.drop('SalePrice')
    lr = linear_model.LinearRegression()
    
    if k==0:
        train = df.iloc[:1460]
        test = df.iloc[1460:]
        lr.fit(train[features], train['SalePrice'])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test['SalePrice'], predictions)
        rmse = np.sqrt(mse)
    elif k ==1:
        # Randomize *all* rows (frac=1) from `df` and return
        shuffled_df = df.sample(frac=1, )
        train = df.iloc[:1460]
        test = df.iloc[1460:]
        
        lr.fit(train[features], train['SalePrice'])
        predictions_1 = lr.predict(test[features])
        mse = mean_squared_error(test['SalePrice'], predictions_1)
        rmse_1 = np.sqrt(mse)
        
        lr.fit(test[features], test['SalePrice'])
        predictions_2 = lr.predict(train[features])
        mse = mean_squared_error(train['SalePrice'], predictions_2)
        rmse_2 = np.sqrt(mse)
        rmse = np.mean([rmse_1,rmse_2])
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse

df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df, k=4)

rmse

[27173.742312000933, 37087.10978129911, 26696.003661475115, 25721.172473389197]


29169.50705704109